<a href="https://colab.research.google.com/github/Rock-Shell/cloud-removal/blob/main/cloud_remove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2><center>Random Submission for Clouds Removal</center></h2>

![](https://media.discordapp.net/attachments/703945836614123561/866939997021143040/banner.jpg?width=1439&height=277)

**Note : Create a copy of the notebook and use the copy for submission. Go to *File > Save a Copy in Drive* to create a new copy**

# Setting up Environment

### Downloading Dataset

So we will first need to download the python library by AIcrowd that will allow us to download the dataset by just inputting the API key. 

In [1]:
!pip install aicrowd-cli

%load_ext aicrowd.magic

     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 170 kB 12.1 MB/s 
     |████████████████████████████████| 62 kB 945 kB/s 
     |████████████████████████████████| 208 kB 53.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requ

In [2]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/9gY3-p3T0j8EqolyOYo-tUd3g-CT9MxdB7jbm1Jyi4Q
API Key valid
Saved API Key successfully!


In [3]:
# Downloading the Dataset
!rm -rf data
!mkdir data
!aicrowd dataset download -c clouds-removal "*Partial*" -o data

test.zip: 100% 601M/601M [00:39<00:00, 15.1MB/s]
train.zip: 100% 1.62G/1.62G [02:08<00:00, 12.6MB/s]


In [4]:
# Unzipping the dataset
!unzip data/train.zip -d data/train >> /dev/null
!unzip data/test.zip -d data/test >> /dev/null

# Importing Libraries

In [8]:
# Importing Libraries
import os
from natsort import natsorted
from glob import glob
import cv2
import numpy as np
from tqdm.notebook import tqdm
import torch.nn as nn
import torch
import torch.nn.functional as fn

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(27, 27)  # One in and one out
        self.tanh = torch.nn.Tanh()
    def forward(self, x):
        y_pred = self.linear(x)
        y_pred = self.tanh(y_pred)
        return y_pred

In [10]:
kernel = np.ones((27),np.uint8)
kernel = torch.from_numpy(kernel.astype(np.float32))#.view(-1,1))

model = Model()

# Define loss function and optimizer function
learning_rate=0.0001
l = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
i =  0

train ="/content/data/train"
cloud = 'cloud_'
clear = 'clear_'

for i in range(500):
  i+=1

  cap = cv2.VideoCapture(train+'/'+clear+str(i)+'.mp4')
  cap2 = cv2.VideoCapture(train+'/'+cloud+str(i)+'.mp4')

  ret,frame = cap.read()
  ret2,frame2 = cap2.read()
  # print(cap.isOpened())
  j=0
  while cap.isOpened():
      j+=1
      ret,frame = cap.read() # clear
      ret2,frame2 = cap2.read() # cloud
      if isinstance(frame,type(None)):
          break
      
      optimizer.zero_grad()
      kernel = model.forward(kernel.requires_grad_())
      Filter = kernel.detach().numpy()
      Filter = Filter.reshape(3,3,3)
      FB,FG,FR = Filter

      frame = torch.from_numpy(frame).type(torch.FloatTensor)
      
      b,g,r = frame2[:,:,0], frame2[:,:,1], frame2[:,:,2]
      b, g, r = cv2.filter2D(b,-1,FB), cv2.filter2D(g,-1,FG), cv2.filter2D(r,-1,FR)
      frame1 = cv2.merge([b,g,r]) # output filtered image
      
      frame1 = torch.from_numpy(frame1).type(torch.FloatTensor).requires_grad_()
      loss = l(frame1,frame)
      if i%25==0 and j==23:
        print(f'loss:{i} {loss}')
      loss.backward()
      optimizer.step()
      if cv2.waitKey(1)==27:
          break
  # if i%25==0:
    # print(f'loss:{i}')
  cap.release()
  cap2.release()

loss:50 11450.26171875
loss:75 15003.2529296875
loss:100 17694.30859375
loss:125 9606.0966796875
loss:150 5103.83837890625
loss:175 4338.53369140625
loss:200 11558.5634765625
loss:225 7769.88818359375
loss:250 11280.3134765625
loss:275 13841.3564453125
loss:300 11649.3916015625
loss:325 12848.1748046875
loss:350 6751.66650390625
loss:375 9939.0517578125
loss:400 16990.486328125
loss:425 11682.7763671875
loss:450 9768.041015625
loss:475 4134.67919921875


In [27]:
kernel

tensor([-0.1990, -0.0812, -0.0327, -0.0846, -0.1567, -0.1371,  0.2515,  0.0787,
         0.0307,  0.0964, -0.0152, -0.0546, -0.0600, -0.1793, -0.1910,  0.1022,
        -0.1493, -0.0375,  0.0296, -0.0073, -0.0418, -0.0221,  0.0033, -0.0980,
         0.0043, -0.2218,  0.1132], grad_fn=<TanhBackward>)

# Generate Random Submission

In this section we will be generating a random submission. We will read all of the files from the tesing directroy and save the same video in `clear` directory for submsision. 

In [28]:
# Creating a clear directory
!rm -rf clear
!mkdir clear

In [ ]:
# Random submission function
def random_submission(data_directory):

  # List of all videos
  video_files = natsorted(glob(data_directory))

  # Groung through each video
  for idx, img_file in enumerate(tqdm(video_files)):
    
    # Saving a new video file 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join("clear", f"clear_{idx}.mp4"), fourcc, 24.0, (512,512))

    # Reading the video
    img_video = cv2.VideoCapture(img_file)

    # Going through each frame
    ret=True
    while ret:
      
      # Reading the frame
      ret, frame = img_video.read()

      # ANY PREPROCESSIG FUNCTION

      # Adding the frame to video writer
      out.write(frame)

In [ ]:
# Running the function
random_submission("data/test/cloud*")

  0%|          | 0/500 [00:00<?, ?it/s]

# Submitting Results 📄

## Uploading the Results

In [ ]:
!aicrowd notebook submit -c clouds-removal -a clear --no-verify

Mounting Google Drive 💾
Your Google Drive will be mounted to access the colab notebook
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWhUnFUF-s1RIO9qTipri-ETF1fGbXJdDNNztbMfeZ-gmoh_e3RY4Gk
Mounted at /content/drive
Using notebook: /content/drive/MyDrive/Colab Notebooks/[Random Submission] Cloud Removal for submission...
Scrubbing API keys from th

> Don't be shy to ask question related to any errors you are getting or doubts in any part of this notebook in [discussion forum](https://www.aicrowd.com/challenges/ai-blitz-10/problems/clouds-removal/discussion) or in [AIcrowd Discord sever](https://discord.gg/T6uZSWBMSZ), AIcrew will be happy to help you :)

Also, wanna give us your valuable feedback for next blitz or wanna work with us creating blitz challanges ? Let us know! 